In [7]:
"""
    Denne koden henter laksedata fra SSB og bruker Plotly til å lage en interaktiv visualisering med dobbel akse. 
    Det er brukt to y-akser for å illustrere sammenhengen tyelig
    Grafene viser gjennomsnittlig årlig utvikling i vekt (tonn) og kilopris (kr/kg) for fersk laks fra 2000 til 2025.
"""

import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go

# Hent data fra SSB API
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {
    "query": [],
    "response": {
        "format": "json-stat2"
    }
}
response = requests.post(api_url, json=query)

if response.status_code == 200:
    data = response.json()

    #Trekker ut dimensjoner og verdier
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values

    #Filtrer ut relevante rader for fersk og frosset laks
    df_pivot = df.pivot_table(
        index="Tid",
        columns=["VareGrupper2", "ContentsCode"],
        values="Value",
        aggfunc="sum"
    ).reset_index()

    #Bytter ut og gir lesbare og forståelig navn på kolonnene 
    df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                        "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

    # Hent ut år fra Tid (første 4 tegn)
    df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)

    #sorterer data per år og regner ut gjennomsnitt av alle numeriske kolonner per år 
    df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)

    #Lager en interaktiv graf med Plotly 
    fig = go.Figure()

    #Legger til en graf for fersk laks vekt og kilospris
    fig.add_trace(go.Scatter(
        x=df_agg["År"],
        y=df_agg["Fersk laks - Vekt (tonn)"],
        name="Fersk laks - Vekt (tonn)",
        yaxis="y1",
        mode="lines+markers"
    ))

    fig.add_trace(go.Scatter(
        x=df_agg["År"],
        y=df_agg["Fersk laks - Kilospris"],
        name="Fersk laks - Kilospris (kr/kg)",
        yaxis="y2",
        mode="lines+markers"
    ))
    #Legger til layout for dobbel akse
    fig.update_layout(
        title="Fersk laks - Årlig gjennomsnitt av vekt og kilospris",
        xaxis=dict(title="År"),
        yaxis=dict(title="Vekt (tonn)", side="left"),
        yaxis2=dict(title="Kilospris (kr/kg)", overlaying="y", side="right"),
        legend=dict(x=0.01, y=0.99),
        hovermode="x unified",
        template="plotly_white",
        height=600
    )

    fig.show() #Viser grafen

else:
    print(f"Feil ved henting av data: {response.status_code}")
    print(response.text)


Grafen viser årlige gjennomsnittlige verdier av kilopris og eksportert vekt av fersk laks per år.
**Blå graf** viser gjennomsnittlig eksportert mengde av fersk laks i tonn.
**Rød linje** viser gjennomsnittlig kilopris  for fersk laks i kr/kg.
Vi bruker to y-akser for å kunne sammenligne pris og volum i samme graf, selv om de har ulike måleenheter. 
Dette viser sammenhengen mellom mengde og pris der vi for eksempel kan se om høy eksportmengde av laks påvirker kiloprisen.


In [8]:
""" 
    Denne koden henter fersk laks data fra SSB API og visualiserer den i en interaktiv graf med Plotly.
    Grafen viser en sammenheng mellom eksportverdi, vekt og kilospris for fersk laks over tid. 
    Ved å kombinere data fra både SSB API og en CSV-kilde kan vi analysere hvordan markedsverdien blir påvirket i forhold til vekt og kilospris for fersk laks. 
"""



import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go

# --- LES CSV-FILEN ---
csv_data = pd.read_csv("eksportverdi-laks-hittil-i-år-siste-10-år.csv")
csv_data.columns = ["År", "Eksportverdi (mrd NOK)"]
csv_data["År"] = csv_data["År"].astype(int)

csv_data["Eksportverdi (mill NOK)"] = csv_data["Eksportverdi (mrd NOK)"] * 1000


# --- HENT OG BEHANDLE SSB-DATA ---
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {"query": [], "response": {"format": "json-stat2"}}
response = requests.post(api_url, json=query)

if response.status_code == 200:
    data = response.json()
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values

    df_pivot = df.pivot_table(
        index="Tid",
        columns=["VareGrupper2", "ContentsCode"],
        values="Value",
        aggfunc="sum"
    ).reset_index()

    df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                        "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

    df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)
    df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)

    # --- FLETT DATAENE ---
    df_agg = df_agg[df_agg["År"] >= 2015]  # Behold kun data fra og med 2015

    merged = pd.merge(df_agg, csv_data, on="År", how="left")

    # --- INTERAKTIV GRAF ---
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=merged["År"],
        y=merged["Fersk laks - Vekt (tonn)"],
        name="Fersk laks - Vekt (tonn)",
        yaxis="y1",
        mode="lines+markers"
    ))

    fig.add_trace(go.Scatter(
        x=merged["År"],
        y=merged["Fersk laks - Kilospris"],
        name="Fersk laks - Kilospris (kr/kg)",
        yaxis="y2",
        mode="lines+markers"
    ))

    fig.add_trace(go.Scatter(
        x=merged["År"],
        y=merged["Eksportverdi (mrd NOK)"],
        name="Eksportverdi hittil i år (mrd NOK)",
        yaxis="y3",
        mode="lines+markers"
    ))

    fig.update_layout(
        title="Fersk laks – Vekt, Kilospris og Eksportverdi (per år)",
        xaxis=dict(title="År"),
        yaxis=dict(title="Vekt (tonn)", side="left", position=0.0),
        yaxis2=dict(title="Kilospris (kr/kg)", overlaying="y", side="right", position=1.0),
        yaxis3=dict(title="Eksportverdi (mrd NOK)", overlaying="y", anchor="free", side="right", position=0.85),
        legend=dict(x=0.01, y=0.99),
        hovermode="x unified",
        template="plotly_white",
        height=650
    )

    fig.show()


""" 
    Her brukes en csv fil og ikke api nøkkel for eksportverdidata siden dataen vi fant ikke hadde noen api nøkkel. 
    Dette gjør at dataene ikke blir oppdatert i sanntid og kan derfor utdateres. 
    Api gjør at dataene blir oppdatert i sanntid og er derfor bedre å bruke enn csv fil.
    Hvis dataene skal brukes i virkeligheten bør analysen ha fått inn oppdaterte tall og verdier.
"""

' \n    Her brukes en csv fil og ikke api nøkkel for eksportverdidata siden dataen vi fant ikke hadde noen api nøkkel. \n    Dette gjør at dataene ikke blir oppdatert i sanntid og kan derfor utdateres. \n    Api gjør at dataene blir oppdatert i sanntid og er derfor bedre å bruke enn csv fil.\n    Hvis dataene skal brukes i virkeligheten bør analysen ha fått inn oppdaterte tall og verdier.\n'

Denne grafen viser tre ulike variabler som ser sammenhengen mellom vekt, kilopris og eksportverdi fra 2015 og til 2024
**Kilospris**: Prisen per kilo på laksen
**Vekt**: Hvor mye laks som eksporteres
**Eksportverdi (mrd NOK)**: Den totale verdien av eksporten som er data fra CSV

Vi ser at eksportverdien følger en kombinasjon av pris og mengde.
Når prisene stiger, vil eksportverdien øke, og mengden minker på slutten av grafen.
Når mengden øker, holder kiloprisen seg stabil og eksportverdien øker litt.

Dersom prisene er høye kan denne grafen vise hvordan eksportinntektene for Norge kan øke, selvom det er år der eksportvolumet er lavere.


In [9]:
""" 
    Koden henter fersk laks-data fra SSB API og data fra en CSV-fil som inneholder eksportverdi for laks. 
    Den lager en interaktiv graf av kiloprisen og eksportverdien for fersk laks.
    Den tar i tillegg med en prediksjon av eksportverdien basert på en regresjonsmodell som bruker kiloprisen som input verdi.
    Dette viser hvordan eksportverdien kan forventes å utvikle seg i fremtiden basert på historiske data.
"""

import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np

# --- 1. Les eksportverdi fra CSV ---
csv_data = pd.read_csv("eksportverdi-laks-hittil-i-år-siste-10-år.csv")
csv_data.columns = ["År", "Eksportverdi (mrd NOK)"]
csv_data["År"] = csv_data["År"].astype(int)
csv_data["Eksportverdi (mill NOK)"] = csv_data["Eksportverdi (mrd NOK)"] * 1000

# --- 2. Hent fersk laks-data fra SSB API ---
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {"query": [], "response": {"format": "json-stat2"}}
response = requests.post(api_url, json=query)

if response.status_code != 200:
    raise Exception(f"SSB API-feil: {response.status_code}")

data = response.json()
dimension_names = list(data["dimension"].keys())
values = data["value"]
dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
all_combinations = list(product(*dimensions))

df = pd.DataFrame(all_combinations, columns=dimension_names)
df["Value"] = values

df_pivot = df.pivot_table(
    index="Tid",
    columns=["VareGrupper2", "ContentsCode"],
    values="Value",
    aggfunc="sum"
).reset_index()

df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                    "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)
df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)
df_agg = df_agg[df_agg["År"] >= 2015]

# --- 3. Flett sammen eksportverdi og laksedata ---
merged = pd.merge(df_agg, csv_data[["År", "Eksportverdi (mill NOK)"]], on="År", how="left")
filtered = merged.dropna(subset=["Eksportverdi (mill NOK)", "Fersk laks - Kilospris"])

# --- 4. Regresjonsmodell: Kilospris → Eksportverdi ---
X = filtered[["Fersk laks - Kilospris"]]
y = filtered["Eksportverdi (mill NOK)"]
model = LinearRegression()
model.fit(X, y)

# Lag framtidige kilospriser (projiserte verdier) fra 2026 til 2035
# For enkelhet bruker vi gjennomsnittlig pris + små endringer
framtid_kilospris = np.linspace(X["Fersk laks - Kilospris"].min(), X["Fersk laks - Kilospris"].max() + 5, 10)
framtid = pd.DataFrame({"Fersk laks - Kilospris": framtid_kilospris})
framtid["Predikert eksportverdi"] = model.predict(framtid)




#Kombinert interaktiv graf
fig = go.Figure()

# Kilospris over tid
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Fersk laks - Kilospris"],
    name="Kilospris (kr/kg)",
    yaxis="y1",
    mode="lines+markers"
))

# Eksportverdi over tid
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Eksportverdi (mill NOK)"],
    name="Eksportverdi (mill NOK)",
    yaxis="y2",
    mode="lines+markers"
))

# Predikert eksportverdi basert på kilospris
fig.add_trace(go.Scatter(
    x=framtid["Fersk laks - Kilospris"],
    y=framtid["Predikert eksportverdi"],
    name="Predikert eksportverdi",
    xaxis="x2",  # Bruk sekundær x-akse
    yaxis="y2",
    mode="lines+markers",
    line=dict(dash="dot", color="orange")
))

# Layout med to x-akser og to y-akser
fig.update_layout(
    title="Fersk laks – Historisk og predikert eksportverdi",
    xaxis=dict(
        title="År",
        domain=[0, 0.7]
    ),
    xaxis2=dict(
        title="Kilospris (kr/kg)",
        domain=[0.75, 1],
        anchor="y2"
    ),
    yaxis=dict(
        title="Kilospris (kr/kg)",
        side="left"
    ),
    yaxis2=dict(
        title="Eksportverdi (mill NOK)",
        side="right",
        overlaying="y"
    ),
    legend=dict(x=0.01, y=0.99),
    template="plotly_white",
    height=650
)

fig.show()

Denne visualiseringe viser:
- Historisk utvikling i kilopris og eksportverdi for fersk laks (2015–2025)
- En prediktiv modell som estimerer fremtidig eksportverdi basert på kilospris (2026–2035).

- Det er brukt lineær regresjon der mellom variabelen "fersk laks - Kilospris" og variabelen "Eksportverdi i mill. NOK".
- Prediksjonen (oransje graf) viser hvordan eksportverdi kan utvikle seg hvis kiloprisen stiger jevnt i årene fremover.
- Modellen er trent på reelle data men viser en forenklet trend for fremtiden.


- Dette gir en verdifull innsikt i hvordan prisnivået alene kan påvirke eksportverdien, men forutsatt at volumet er stabilt.
- Modellen tar ikke hensyn til valutakurser, etterspørsel eller politikk.
- Ved å bruke "scikit-learn og Plotly" får vi analysert i detaljer og det gir gode og oversiktlige grafer.

- Kilden for historiske eksportverdier er en CSV-fil som ikke oppdateres automatisk.
- Vi brukte "LinearRegression()" fra "sklearn" og "np.linspace()" til å generere fremtidige prisnivåer.




In [10]:
"""
    Denne koden kombinerer API-data fra SSB og CSV-data om eksportverdi.
    Koden trener en lineær regresjonsmodell for å forutsi fremtidig eksportverdi basert på kilopris for fersk laks, og visualiserer sammenhengen og prediksjonen
    i en interaktiv graf med dobbel y-akse.
    Dette viser sammenhengen mellom kilospris og eksportverdi både historisk og fremover i tid.
"""




import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np

# --- 1. Les eksportverdi fra CSV ---
csv_data = pd.read_csv("eksportverdi-laks-hittil-i-år-siste-10-år.csv")
csv_data.columns = ["År", "Eksportverdi (mrd NOK)"]
csv_data["År"] = csv_data["År"].astype(int)
csv_data["Eksportverdi (mill NOK)"] = csv_data["Eksportverdi (mrd NOK)"] * 1000

# --- 2. Hent fersk laks-data fra SSB API ---
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {"query": [], "response": {"format": "json-stat2"}}
response = requests.post(api_url, json=query)
data = response.json()

# Pakk ut data
dimension_names = list(data["dimension"].keys())
values = data["value"]
dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
all_combinations = list(product(*dimensions))

df = pd.DataFrame(all_combinations, columns=dimension_names)
df["Value"] = values

# Pivot tabellen
df_pivot = df.pivot_table(
    index="Tid",
    columns=["VareGrupper2", "ContentsCode"],
    values="Value",
    aggfunc="sum"
).reset_index()

df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                    "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)
df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)
df_agg = df_agg[df_agg["År"] >= 2015]

# --- 3. Flett dataene ---
merged = pd.merge(df_agg, csv_data[["År", "Eksportverdi (mill NOK)"]], on="År", how="left")
filtered = merged.dropna(subset=["Eksportverdi (mill NOK)", "Fersk laks - Kilospris"])

# --- 4. Tren regresjonsmodell ---
X = filtered[["Fersk laks - Kilospris"]]
y = filtered["Eksportverdi (mill NOK)"]

model = LinearRegression()
model.fit(X, y)

# --- 5. Lag prediksjon for fremtidige år ---
framtid_år = list(range(filtered["År"].max() + 1, filtered["År"].max() + 11))
siste_pris = filtered["Fersk laks - Kilospris"].iloc[-1]
prisvekst = 0.5  # kr/kg økning per år

framtid = pd.DataFrame({
    "År": framtid_år,
    "Fersk laks - Kilospris": [siste_pris + prisvekst * i for i in range(1, 11)]
})
framtid["Predikert eksportverdi"] = model.predict(framtid[["Fersk laks - Kilospris"]])

# --- 6. Kombinert graf med to y-akser ---
fig = go.Figure()

# Historisk kilospris
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Fersk laks - Kilospris"],
    name="Kilospris (kr/kg)",
    yaxis="y1",
    mode="lines+markers"
))

# Historisk eksportverdi
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Eksportverdi (mill NOK)"],
    name="Eksportverdi (mill NOK)",
    yaxis="y2",
    mode="lines+markers"
))

# Predikert eksportverdi
fig.add_trace(go.Scatter(
    x=framtid["År"],
    y=framtid["Predikert eksportverdi"],
    name="Predikert eksportverdi",
    yaxis="y2",
    mode="lines+markers",
    line=dict(dash="dot", color="orange")
))

# Oppsett
fig.update_layout(
    title="Fersk laks – Kilospris og eksportverdi med prediktiv analyse",
    xaxis=dict(title="År"),
    yaxis=dict(title="Kilospris (kr/kg)", side="left"),
    yaxis2=dict(title="Eksportverdi (mill NOK)", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99),
    template="plotly_white",
    height=650
)

fig.show()

Prisen og eksportverdien henger sammen fordi når kiloprisen går opp, ser vi også at eksportverdien går opp. Dette kan skyldes høyere etterspørsel, bedre kvalitet eller lavere tilbud.
Modellen sin prediksjon antar at det er en jevn prisvekst. Den er basert på litt prisøkning hvert år og er en forenklet modell fordi vi vet at verdien vil svinge enda mer enn den linære linjen viser.

Modellen tar ikke hensyn til valutakurser, produksjonsvolum, globale markeder eller verdenssituasjonen.
Grafen kan brukes til å gi et grovt estimat på fremtidig eksportverdi, og illustrerer hvordan pris påvirker inntektene fra eksport.